# RAG Project - General Handbook / Querying Database and Prompting LLM
## Querying Pinecone
I will start by importing the required libraries.

In [1]:
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai

from dotenv import load_dotenv
import os
load_dotenv()

## Initializing connection to Pinecone index
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("general-handbook")

## Initializing Google API to generate embeddings
google_api_key=os.environ.get("GOOGLE_API_KEY")

genai.configure(api_key=google_api_key)

/home/angelo/ai-internship/personal-ai-project/.venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Now, I will create a function that sends queries to Pinecone and returns the response.

In [2]:
def query_pinecone(query_text, top_k=5):
    """
    This function queries the Pinecone index for the given query text and returns the top-k results as a list of formatted strings.
    args:
        query_text: string
        top_k: integer
    """
    
    query_vector = genai.embed_content(
        model = "models/embedding-001",
        content = query_text,
        task_type = "retrieval_document",
        title = "Embedding of single string"
    )

    results = index.query(
        namespace = "general-handbook-vectors",
        vector = query_vector["embedding"],
        top_k = top_k,
        include_values = False,
        include_metadata = True,
        # filter = {"genre": {"$eq": "action"}}
    )

    match_list = []

    for matched in results["matches"]:
        match_string = f"""
        Chapter: {matched['metadata']['chapter']}
        Header: {matched['metadata']['title']}
        Section: {matched['metadata']['section']}
        Url: {matched['metadata']['url']}
        --------------------------------------------------
        Content: {matched['metadata']['content']}
        """

        match_list.append(match_string)

        match_string = "\n".join(match_list)

    return match_string

In [20]:
matches = query_pinecone("What are the responsibilities of the Elder's Quorum President?", top_k = 5)

print(matches)


        Chapter: Elders Quorum
        Header: Stake Presidency and Bishop
        Section: 8.3
        Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng
        --------------------------------------------------
        Content: The elders quorum president is directly responsible to the stake presidency. He meets regularly with a member of the presidency to receive direction and report on his responsibilities. The elders quorum president also receives guidance from the bishop, who is the presiding officer in the ward. They meet regularly. They discuss their efforts in God’s work of salvation and exaltation, including the service of ministering brothers. They also discuss (1) the progress and needs of ward members and (2) elders quorum meetings, instruction, and activities.
        

        Chapter: Elders Quorum
        Header: Activities
        Section: 8.2
        Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/

I will create a second version of this function that uses a context window instead, and returns a larger chunk of information.

In [34]:
import textwrap

def query_pinecone_with_id_reference(query_text, top_k=5, context_window=3):
    """
    Query Pinecone with the given query text and return a context window based on the reference vector's ID.
    
    Args:
        query_text (str): The query text to search.
        top_k (int): Number of top results to return.
        context_window (int): Number of surrounding vectors to include before and after the reference vector.
        
    Returns:
        list: A list of formatted strings with the reference metadata at the top and the surrounding content.
    """
    
    # Step 1: Embed the query text using Gemini API
    query_vector = genai.embed_content(
        model="models/embedding-001",
        content=query_text,
        task_type="retrieval_document",
        title="Embedding of single string"
    )

    # Step 2: Query Pinecone for the top result (reference vector)
    results = index.query(
        namespace="general-handbook-vectors",
        vector=query_vector["embedding"],
        top_k=top_k,  # Retrieve just the top result
        include_values=False,
        include_metadata=True
    )

    # Step 3: Find the top result (reference vector) and its ID
    top_result = results["matches"][0]
    reference_id = int(top_result["id"])  # Convert the ID to an integer
    
    # Step 4: Retrieve the exact preceding and following vectors using the reference ID
    surrounding_ids = list(range(reference_id - context_window, reference_id + context_window + 1))

    # Step 5: Query Pinecone for each surrounding vector by ID
    context_matches = []
    for vector_id in surrounding_ids:
        try:
            result = index.query(
                namespace="general-handbook-vectors",
                id=str(vector_id),  # Use the vector ID as a string
                top_k=1,  # Get just the vector with this ID
                include_values=False,
                include_metadata=True
            )
            context_matches.append(result["matches"][0])  # Add the result to the context matches list
        except:
            # If a vector with the ID doesn't exist, skip it
            continue

    # Step 6: Format the metadata for the reference vector
    reference_metadata = textwrap.dedent(f"""
    Reference Metadata:
    Chapter: {top_result['metadata']['chapter']}
    Header: {top_result['metadata']['title']}
    Section: {top_result['metadata']['section']}
    Url: {top_result['metadata']['url']}
    --------------------------------------------------""")

    # Step 7: Format the surrounding context (excluding metadata, just content)
    preceding_context = []
    following_context = []
    
    for match in context_matches:
        if match["id"] == str(reference_id):
            continue  # Skip the reference vector itself
        elif int(match["id"]) < reference_id:
            preceding_context.append(textwrap.dedent(f"Content: {match['metadata']['content']}"))
        else:
            following_context.append(textwrap.dedent(f"Content: {match['metadata']['content']}"))

    # Step 8: Combine the preceding context, reference metadata, and following context
    final_output = [reference_metadata] + preceding_context + [f"Content: {top_result['metadata']['content']}"] + following_context
    print("Top result: ", top_result["metadata"]["content"])

    match_string = "\n".join(final_output)

    return match_string


In [23]:
matches = query_pinecone_with_id_reference("What are the responsibilities of the Elder's Quorum President?", top_k = 5, context_window = 3)

print(matches)


Reference Metadata:
Chapter: Elders Quorum
Header: Stake Presidency and Bishop
Section: 8.3
Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng
--------------------------------------------------
Content: Melchizedek Priesthood holders have a duty to invite all to receive the blessings of the gospel of Jesus Christ (see Doctrine and Covenants 53:3). They do this by sharing the gospel, strengthening new members, and reaching out to those who are less active. See 23.2 and 23.3. In presidency and quorum meetings, the elders quorum presidency discusses ways the brethren can invite all to receive the blessings of the gospel. See 23.1.1, 23.1.2, and 23.1.3. The elders quorum president assigns a member of the presidency to help lead efforts to share the gospel in the ward. He works with the assigned member of the Relief Society presidency to coordinate these efforts (see 23.6.2). This member of the elders quorum presidency may fulfill the role of th

## Prompting Gemini with queried vectors
Now, I will create a function that takes up the results from the query to Pinecone, and uses them to answer the user's question.

First, let's import and create the necessary funcitons for the initial setup.

In [24]:
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

text_generator = genai.GenerativeModel('gemini-1.5-flash')

I will first try with a test prompt, and adjust it before creating the funtions that will put it all together.

In [17]:
promt = """
Your role:
You are a chatbot that answers questions about the General Handbook of the Church of Jesus Christ of Latter Day Saints.

Intructions:
Below is a list of information received from a vector database where you might find information to answer the user's question about the General Handbook.
Each chunk of information contains a title, url, and content. You will mainly answer questions from the content, but feel free to share that extra information as reference.

Information:
START OF INFORMATION

Chapter: Elders Quorum
Header: Stake Presidency and Bishop
Section: 8.3
Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng
--------------------------------------------------
Content: The elders quorum president is directly responsible to the stake presidency. He meets regularly with a member of the presidency to receive direction and report on his responsibilities. The elders quorum president also receives guidance from the bishop, who is the presiding officer in the ward. They meet regularly. They discuss their efforts in God’s work of salvation and exaltation, including the service of ministering brothers. They also discuss (1) the progress and needs of ward members and (2) elders quorum meetings, instruction, and activities.


Chapter: Elders Quorum
Header: Activities
Section: 8.2
Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng
--------------------------------------------------
Content: Elders quorum presidencies may plan activities. These activities strengthen quorum members and give them opportunities to serve together. Most activities are held at times other than on Sundays or Monday evenings. The elders quorum president oversees these activities. He may ask a counselor or another quorum member to take the lead in planning and carrying them out (see 8.3.5). Presidencies discuss activities with the bishop as part of their planning.


Chapter: Elders Quorum
Header: High Councilor
Section: 8.3
Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng
--------------------------------------------------
Content: The stake presidency assigns a high councilor to represent them in each elders quorum. His responsibilities are outlined in 6.5.


Chapter: Sharing the Gospel and Strengthening New and Returning Members
Header: High Councilors
Section: 23.5
Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/23?lang=eng
--------------------------------------------------
Content: The stake presidency may assign high councilors to instruct and support elders quorum presidencies and ward mission leaders. One or more high councilors may be assigned to lead these efforts. However, all high councilors have these responsibilities for the wards and quorums to which they are assigned.


Chapter: Elders Quorum
Header: Living the Gospel of Jesus Christ
Section: 8.2
Url: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng
--------------------------------------------------
Content: Elders quorum leaders support members in living the gospel of Jesus Christ. They emphasize exercising faith, repenting, increasing in self-reliance, and making covenants with God by receiving ordinances. Such ordinances include the temple endowment. See 1.2.1.

END OF INFORMATION

(Further instructions: If the user seems to be asking about something not related to the General Handbook, invite them to ask about it. If instead of a questions they seem to be thanking you for previous answers, show that you welcome their gratitude)
User Input:
What are the responsibilities of the Elders' Quorum Presidency?
"""

response = text_generator.generate_content(promt)

to_markdown(response.text)

> According to the General Handbook of the Church of Jesus Christ of Latter-day Saints, the elders quorum president is directly responsible to the stake presidency and meets regularly with a member of the presidency to receive direction and report on his responsibilities.  The elders quorum president also receives guidance from the bishop, who is the presiding officer in the ward, and they meet regularly.  They discuss their efforts in God's work of salvation and exaltation, including the service of ministering brothers, the progress and needs of ward members, and elders quorum meetings, instruction, and activities. The elders quorum presidencies may also plan activities to strengthen quorum members and give them opportunities to serve together.  These activities are overseen by the elders quorum president and are most often held at times other than on Sundays or Monday evenings.  They also discuss activities with the bishop as part of their planning. You can find this information in the General Handbook under the following URL: https://www.churchofjesuschrist.org/study/manual/general-handbook/8-elders-quorum?lang=eng. 


Finally, I will use gradio as my UI, and I will use the query_pinecone function and the generate_content function to prompt Gemini for answers to the users questions.

In [ ]:
import random
import gradio as gr

def format_chat_prompt(message, chat_history):
    prompt = ""
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history):
        
        #information = query_pinecone(message)        
        information = query_pinecone_with_id_reference(message, top_k=7, context_window=5)
        print(information)

        full_prompt = f"""
        Your role:
        You are a chatbot that answers questions about the General Handbook of the Church of Jesus Christ of Latter Day Saints.

        Intructions:
        Below is a list of information received from a vector database where you might find information to answer the user's question about the General Handbook.
        Each chunk of information contains a title, url, and content. You will mainly answer questions from the content, but feel free to share that extra information as reference.

        Information:
        START OF INFORMATION

        {information}

        END OF INFORMATION

        (Further instructions: If the user seems to be asking about something not related to the General Handbook, invite them to ask about it. If instead of a questions they seem to be thanking you for previous answers, show that you welcome their gratitude.)
        User Input:
        {message}
        """


        formatted_prompt = format_chat_prompt(full_prompt, chat_history)
        bot_message = text_generator.generate_content(formatted_prompt).text
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480) #just to fit the notebook
    msg = gr.Textbox(label="Question")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch(share=True)